In [41]:
import ipynb
import torch
from ipynb.fs.full.data_handler import load_subject, extract_labels, extract_action_interval, split_data, split_info, to_device, get_innerspeech
from ipynb.fs.full.train_model import train_model, accuracy_check

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

data, description = load_subject(1)
data, description = get_innerspeech(data, description)
labels = extract_labels(description)
#data_interval = extract_action_interval(data)

train_data, val_data, test_data, train_labels, val_labels, test_labels = split_data(data, labels)

train_data, val_data, test_data, train_labels, val_labels, test_labels = to_device(train_data, val_data, test_data, train_labels, val_labels, test_labels, device)
split_info(train_data, val_data, test_data, train_labels, val_labels, test_labels)

[39, 39, 40, 40] 
 [6, 6, 5, 5] 
 [5, 5, 5, 5]
24.68354430379747 %  24.68354430379747 %  25.31645569620253 %  25.31645569620253 % 
27.27272727272727 %  27.27272727272727 %  22.727272727272727 %  22.727272727272727 % 
25.0 %  25.0 %  25.0 %  25.0 % 


# Deep CNN
Implementation of the deep CNN structure from (Schirrmeister et. al.) for the use on "Thinking out loud" dataset.

In [42]:
import torch
import torch.nn as nn
import torch.optim as optim

class DeepCNN(nn.Module):
    def __init__(self, hz, interval = "full", dropout = 0.5):
        super(DeepCNN, self).__init__()
        first_channels = 25
        second_channels = 50
        third_channels = 100
        fourth_channels = 200

        ### Block 1
        # Temporal convolution
        self.tempconv = nn.Conv2d(in_channels = 1, out_channels = first_channels, kernel_size = (1, 10), padding = 0, bias = False)
        # Spatial convolution
        self.spatconv = nn.Conv2d(in_channels = first_channels, out_channels = first_channels, kernel_size=(128,1), padding = 0, bias = False)
        # Batch normalization
        self.batchnorm1 = nn.BatchNorm2d(first_channels, False)
        # ELU
        self.elu = nn.ELU()
        # Dropout
        self.dropout = nn.Dropout(dropout)
        # Max pooling
        self.maxpool = nn.MaxPool2d(kernel_size = (1,3), stride = (1,3))

        ### Block 2
        self.conv2 = nn.Conv2d(in_channels = first_channels, out_channels = second_channels, kernel_size = (1,10), padding = 0, bias = False)
        # Batch normalization
        self.batchnorm2 = nn.BatchNorm2d(second_channels, False)

        ### Block 3
        self.conv3 = nn.Conv2d(in_channels = second_channels, out_channels = third_channels, kernel_size = (1,10), padding = 0, bias = False)
        # Batch normalization
        self.batchnorm3 = nn.BatchNorm2d(third_channels, False)

        ### Block 4
        self.conv4 = nn.Conv2d(in_channels = third_channels, out_channels = fourth_channels, kernel_size = (1,10), padding = 0, bias = False)
        # Batch normalization
        self.batchnorm4 = nn.BatchNorm2d(fourth_channels, False)

        # Classifier
        if interval == "action":
            self.classifier = nn.Linear(600,4, bias = False)
        elif interval == "full":
            self.classifier = nn.Linear(1800,4, bias = False)
        # Softmax
        self.softmax = nn.LogSoftmax(dim = 1)

        
    def forward(self, x):
        # Block 1
        res = self.tempconv(x)
        res = self.spatconv(res)
        res = self.batchnorm1(res)
        res = self.elu(res)
        res = self.maxpool(res)
        res = self.dropout(res)
        # Block 2
        res = self.conv2(res)
        res = self.batchnorm2(res)
        res = self.elu(res)
        res = self.maxpool(res)
        # Block 3
        res = self.conv3(res)
        res = self.batchnorm3(res)
        res = self.elu(res)
        res = self.maxpool(res)
        # Block 4
        res = self.conv4(res)
        res = self.batchnorm4(res)
        res = self.elu(res)
        res = self.maxpool(res)
        # Classifier
        res = torch.flatten(res, start_dim=1)
        res = self.classifier(res)
        res = self.softmax(res)
        return res

In [43]:
network = DeepCNN(interval = "full").float().to(device)
accuracy_check(network, train_data, train_labels)

print("####### TRAINING #######")
op = optim.Adam(params = network.parameters(), lr = 0.0001)
lossf = nn.NLLLoss()
train_model(network, train_data = train_data, train_labels = train_labels, val_data = val_data, val_labels = val_labels, epochs = 50, batch_size = 10, loss_func = lossf, optimizer = op)

accuracy_check(network, train_data, train_labels)

ACCURACY: 0.2721518987341772
####### TRAINING #######
Epoch	 train loss	 validation loss
0 	  1.5352389017740886 	 1.4157025814056396
1 	  1.2211573282877604 	 1.5906962156295776
2 	  1.0767817179361978 	 1.7430084943771362
3 	  0.9756191889444987 	 1.8270012140274048
4 	  0.8629987716674805 	 1.8945465087890625
5 	  0.7639547348022461 	 1.9555039405822754
6 	  0.714982541402181 	 1.9820233583450317
7 	  0.6359867731730143 	 2.02097749710083
8 	  0.5825490315755208 	 2.1104447841644287
9 	  0.531870969136556 	 2.211881399154663
10 	  0.44982865651448567 	 2.3064687252044678
11 	  0.38181320826212567 	 2.324892997741699
12 	  0.3349457740783691 	 2.3447086811065674
13 	  0.2899471282958984 	 2.3505802154541016
14 	  0.24329446156819662 	 2.3297154903411865
15 	  0.220074733098348 	 2.296295404434204
16 	  0.18185375531514486 	 2.3326973915100098
17 	  0.18009146054585776 	 2.3253915309906006
18 	  0.13987733523050944 	 2.4022438526153564
19 	  0.12449454466501872 	 2.4532487392425537
20

In [44]:
accuracy_check(network, train_data, train_labels)
accuracy_check(network, val_data, val_labels)
accuracy_check(network, test_data, test_labels)


ACCURACY: 0.8481012658227848
ACCURACY: 0.2727272727272727
ACCURACY: 0.3
